In [181]:
#import the python request library to query a website
import requests
import pandas as pd
#specify the url we want to scrape from
Link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#convert the web page to text
Link_text = requests.get(Link).text
#print(Link_text)
#import BautifulSoup library to pull data out of HTML and XML files
from bs4 import BeautifulSoup
#to convert Link_text into a BeautifulSoup Object
soup = BeautifulSoup(Link_text, 'lxml')

###### I have failed to identify which table has the data. I have decided to get all tables 

In [182]:
our_table = soup.find_all('table')

#### It seems data i want is in the first table

#### My approach
1. Get data and put it in three different lists
2. Use pandas to series method to convert the lists into pandas data frame
3. Use regex to extract neiborhood data 

In [207]:
import re
row_marker = 0
## create an empty list
list_post=[]  #list of postal coes
list_address=[] # list of neiborhoods
list_borough=[]
for row in our_table[0].find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
            #Get the text
            s=column.get_text().strip()
            #print(s)
            # here i remove where it contains the words 
            if 'Not assigned' not in s:
                ## get post addrees 3 characters
                t=s[:3].strip()
                ## add it to the list.
                list_post.append(t)
                ## get the borough...the text before the blackets
                ## from start to position of the first (
                pos_b=int(s.find('('))
                #print(pos_b)
                if pos_b==-1:
                    borough=s[3:]
                else:
                    borough=s[3:pos_b]
                list_borough.append(borough)
                ##get neiborhood(one in brackets)
                address=re.findall(r'\(.*?\)',s)
                ## remove the brackets
                ## if address is an empty list? use the borough as per instructions
                if len(address)==0:
                    address=borough
                    #print(address)
                else:
                    address=''.join(address).replace('(','').replace(')','')
                    
                list_address.append(address)
                #list_boro.append(s)
            column_marker += 1


#### Below i created the data frame from the lists above

In [218]:
df=pd.DataFrame()
df['Postal Code']=pd.Series((i for i in list_post))
df['Borough']=pd.Series((i for i in list_borough))
df['Neighborhood']=pd.Series((i for i in list_address))


In [219]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park / Ontario Provincial Government,Queen's Park / Ontario Provincial Government


### Are there  neighborhoods with  one postal code area as suggested in the question?. The value counts below suggests otherwise!

In [190]:
df['postalCode'].value_counts()

M6P    1
M7R    1
M9C    1
M1W    1
M8Z    1
M1B    1
M4W    1
M5B    1
M6J    1
M8Y    1
M4R    1
M5K    1
M5H    1
M2J    1
M8X    1
M6C    1
M5L    1
M7Y    1
M1M    1
M5X    1
M2H    1
M4S    1
M5R    1
M5C    1
M1H    1
M3K    1
M9M    1
M4K    1
M2M    1
M2K    1
      ..
M1L    1
M1R    1
M5J    1
M4H    1
M3A    1
M9P    1
M4L    1
M7A    1
M6H    1
M1T    1
M4Y    1
M4N    1
M5G    1
M3J    1
M3M    1
M5T    1
M4B    1
M6S    1
M6R    1
M9R    1
M5M    1
M4E    1
M3C    1
M2L    1
M1N    1
M4V    1
M2P    1
M1E    1
M1C    1
M9B    1
Name: postalCode, Length: 103, dtype: int64

### Last part---printing rows only

In [209]:
print(df.shape[0])

103


### Opted to use the provided data set to take advantage of panda merging functions!

In [210]:
## Got data from link provided into a data frame!
df1=pd.read_csv('https://cocl.us/Geospatial_data')

In [214]:
df1.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [220]:
combined =df.merge(df1)
   

In [222]:
combined.head(5)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Queen's Park / Ontario Provincial Government,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [ ]:
df1.colum

In [223]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans 
import folium # map rendering library

In [224]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [225]:
address = 'Toronto, To'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.653963, -79.387207.


In [227]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(combined['Latitude'], combined['Longitude'], combined['Borough'], combined['Neighborhood']):
    label = '{}, {}'.format(combined, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto